In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import datetime as dt
import dateutil.relativedelta as relativedelta

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func,inspect

In [ ]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model
Base = automap_base()
Base.prepare(engine, reflect=True)

# We can view all of the classes that automap found
Base.classes.keys()

In [ ]:
engine.execute('SELECT * FROM measurement LIMIT 5').fetchall()

In [ ]:
engine.execute('SELECT * FROM station LIMIT 5').fetchall()

In [ ]:
# reflect the tables
Measurement = Base.classes.measurement
Station = Base.classes.station

In [ ]:
# inspect (to see if any tables are left out from automap)
inspector = inspect(engine)
# Collect the names of tables within the database
inspector.get_table_names()

In [ ]:
# inspect col names and type of each table
tables = inspector.get_table_names();
for table in tables:
    columns = inspector.get_columns(table)
    print(table)
    print("------------")
    for column in columns:
        print(column["name"], column["type"])
    print("------------")

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Climate Analysis

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results
# Calculate the date 1 year ago from the last data point in the database
# this also works - not sure how func.max operates on str since measurement.date is string
# latestDate = session.query(Measurement.date,func.max(Measurement.date)).first().date
latestData = session.query(Measurement).order_by(Measurement.date.desc()).first()

# Calculate the date 1 year ago from the last data point in the database
toDate = dt.datetime.strptime(latestData.date, '%Y-%m-%d')
fromDate = toDate - relativedelta.relativedelta(months=12) 

# Perform a query to retrieve the data and precipitation scores
oneYearQuery = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date >= fromDate).order_by(Measurement.date.desc()).all()

# Save the query results as a Pandas DataFrame and set the index to the date column
oneYearDf = pd.DataFrame(oneYearQuery,columns=['Date','Precipitation']).set_index('Date')

# Sort the dataframe by date
oneYearDf = oneYearDf.sort_values(by='Date')
oneYearDf.head(20)

# Use Pandas Plotting with Matplotlib to plot the data
oneYearDf.plot(kind = 'bar',width=8, figsize=(12,8))
plt.locator_params(axis='x', nbins=9)
plt.ylabel("Precipitation (Inches)", size=18)
plt.title("Precipitation Last 12 Months", size=20)
plt.savefig("Images/Precipitation.png")

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
oneYearDf.describe()

In [ ]:
# Design a query to show how many stations are available in this dataset?
allStations = session.query(Station.name.distinct())
allStations.count()

In [ ]:
#  display list of station names
allStationNames = list(np.ravel(allStations.all()))
allStationNames

In [ ]:
# What are the most active stations? (i.e. what stations have the most rows)?
# List the stations and the counts in descending order.
stationTable = session.query(Station.name, Measurement.station, func.count(Measurement.station)).group_by(Measurement.station).\
        join(Station, Measurement.station == Station.station).\
        order_by(func.count(Measurement.station).desc())

#  first row is the most active station
stationTable.first()

In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature of the most active station
# get first active station id
mostActiveStation = stationTable[0][1]
session.query(Station.name,Measurement.station, func.min(Measurement.tobs), func.max(Measurement.tobs), func.avg(Measurement.tobs)).\
            join(Station, Measurement.station == Station.station).\
            filter_by(station = mostActiveStation).all()

In [ ]:
#  We can also grab the most active station id by this method
#  instead of mostActiveStation = stationTable[0][1]
mostActiveStationId = session.query(Measurement.station).group_by(Measurement.station).\
        order_by(func.count(Measurement.station).desc()).first()
mostActiveStationId[0]  

In [ ]:
# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
mostActiveStationOneYear = session.query(Measurement.date, Measurement.tobs)\
.filter(Measurement.station == mostActiveStation).filter(Measurement.date >= fromDate).all()
mostActiveStationOneYearDf= pd.DataFrame(mostActiveStationOneYear,columns=["Date","tobs"]).set_index("Date")

mostActiveStationOneYearDf.plot.hist(figsize=(12,8), bins = 12)
plt.xlabel("Temperature Reported", fontsize=16)
plt.title("Temperature Last 12 Months in " + mostActiveStation, size=20)
plt.savefig("Images/Temperature.png")

## Bonus Challenge Assignment

In [ ]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

# function usage example
print(calc_temps('2012-02-28', '2012-03-05'))

In [ ]:
# Use your previous function `calc_temps` to calculate the tmin, tavg, and tmax 
# for your trip using the previous year's data for those same dates.


In [ ]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title
# Use the average temperature for the y value
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)


In [ ]:
# Calculate the total amount of rainfall per weather station for your trip dates using the previous year's matching dates.
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation



In [ ]:
# Create a query that will calculate the daily normals 
# (i.e. the averages for tmin, tmax, and tavg for all historic data matching a specific month and day)

def daily_normals(date):
    """Daily Normals.
    
    Args:
        date (str): A date string in the format '%m-%d'
        
    Returns:
        A list of tuples containing the daily normals, tmin, tavg, and tmax
    
    """
    
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    return session.query(*sel).filter(func.strftime("%m-%d", Measurement.date) == date).all()
    
daily_normals("01-01")

In [ ]:
# calculate the daily normals for your trip
# push each tuple of calculations into a list called `normals`

# Set the start and end date of the trip

# Use the start and end date to create a range of dates

# Stip off the year and save a list of %m-%d strings

# Loop through the list of %m-%d strings and calculate the normals for each date


In [ ]:
# Load the previous query results into a Pandas DataFrame and add the `trip_dates` range as the `date` index


In [ ]:
# Plot the daily normals as an area plot with `stacked=False`
